# Avaliadores

Em alto nível, um avaliador julga uma invocação de sua aplicação LLM contra um exemplo de referência, e retorna uma pontuação de avaliação.

Nos avaliadores do LangSmith, representamos este processo como uma função que recebe um Run (representando a invocação da aplicação LLM) e um Example (representando o ponto de dados a ser avaliado), e retorna Feedback (representando a pontuação do avaliador da invocação da aplicação LLM).

![Evaluator](../../images/evaluator.png)

Aqui está um exemplo de um avaliador personalizado muito simples que compara a saída de um modelo com a saída esperada no dataset:

In [1]:
from langsmith.schemas import Example, Run

def correct_label(inputs: dict, reference_outputs: dict, outputs: dict) -> dict:
  score = outputs.get("output") == reference_outputs.get("label")
  return {"score": int(score), "key": "correct_label"}

### Avaliação LLM-as-Judge

Avaliadores LLM-as-judge usam LLMs para pontuar a saída do sistema. Para usá-los, você tipicamente codifica as regras/critérios de classificação no prompt do LLM. Eles podem ser livres de referência (ex., verificar se a saída do sistema contém conteúdo ofensivo ou adere a critérios específicos). Ou, eles podem comparar a saída da tarefa com uma referência (ex., verificar se a saída é factualmente precisa em relação à referência).

Aqui está um exemplo de como você pode definir um avaliador LLM-as-judge com saída estruturada

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

In [ ]:
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from pydantic import BaseModel, Field

class Similarity_Score(BaseModel):
    similarity_score: int = Field(description="Pontuação de similaridade semântica entre 1 e 10, onde 1 significa não relacionado e 10 significa idêntico.")

# NOTA: Este é nosso avaliador
def compare_semantic_similarity(inputs: dict, reference_outputs: dict, outputs: dict):
    input_question = inputs["question"]
    reference_response = reference_outputs["output"]
    run_response = outputs["output"]
    
    messages = [
        {   
            "role": "system",
            "content": (
                "Você é um avaliador de similaridade semântica. Compare os significados de duas respostas a uma pergunta, "
                "Resposta de Referência e Nova Resposta, onde a referência é a resposta correta, e estamos tentando julgar se a nova resposta é similar. "
                "Forneça uma pontuação entre 1 e 10, onde 1 significa completamente não relacionado, e 10 significa idêntico em significado."
            ),
        },
        {"role": "user", "content": f"Pergunta: {input_question}\n Resposta de Referência: {reference_response}\n Resposta da Execução: {run_response}"}
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    
    # Extrair pontuação da resposta (simplificado)
    try:
        import re
        score_match = re.search(r'\b([1-9]|10)\b', response)
        if score_match:
            score = int(score_match.group(1))
        else:
            score = 5  # pontuação padrão se não conseguir extrair
    except:
        score = 5
    
    return {"score": score, "key": "similarity"}

Vamos testar isso!

NOTA: Propositalmente fizemos esta resposta errada, então esperamos ver uma pontuação baixa.

In [ ]:
# Do Exemplo do Dataset
inputs = {
  "question": "O LangSmith está nativamente integrado com o LangChain?"
}
reference_outputs = {
  "output": "Sim, o LangSmith está nativamente integrado com o LangChain, assim como com o LangGraph."
}


# Da Execução
outputs = {
  "output": "Não, o LangSmith NÃO está integrado com o LangChain."
}

similarity_score = compare_semantic_similarity(inputs, reference_outputs, outputs)
print(f"Pontuação de similaridade semântica: {similarity_score}")

Você também pode definir avaliadores usando Run e Example diretamente!

In [ ]:
from langsmith.schemas import Run, Example

def compare_semantic_similarity_v2(root_run: Run, example: Example):
    input_question = example["inputs"]["question"]
    reference_response = example["outputs"]["output"]
    run_response = root_run["outputs"]["output"]
    
    messages = [
        {   
            "role": "system",
            "content": (
                "Você é um avaliador de similaridade semântica. Compare os significados de duas respostas a uma pergunta, "
                "Resposta de Referência e Nova Resposta, onde a referência é a resposta correta, e estamos tentando julgar se a nova resposta é similar. "
                "Forneça uma pontuação entre 1 e 10, onde 1 significa completamente não relacionado, e 10 significa idêntico em significado."
            ),
        },
        {"role": "user", "content": f"Pergunta: {input_question}\n Resposta de Referência: {reference_response}\n Resposta da Execução: {run_response}"}
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    
    # Extrair pontuação da resposta (simplificado)
    try:
        import re
        score_match = re.search(r'\b([1-9]|10)\b', response)
        if score_match:
            score = int(score_match.group(1))
        else:
            score = 5  # pontuação padrão se não conseguir extrair
    except:
        score = 5

    return {"score": score, "key": "similarity"}

In [ ]:
sample_run = {
  "name": "Execução de Exemplo",
  "inputs": {
    "question": "O LangSmith está nativamente integrado com o LangChain?"
  },
  "outputs": {
    "output": "Não, o LangSmith NÃO está integrado com o LangChain."
  },
  "is_root": True,
  "status": "success",
  "extra": {
    "metadata": {
      "key": "value"
    }
  }
}

sample_example = {
  "inputs": {
    "question": "O LangSmith está nativamente integrado com o LangChain?"
  },
  "outputs": {
    "output": "Sim, o LangSmith está nativamente integrado com o LangChain, assim como com o LangGraph."
  },
  "metadata": {
    "dataset_split": [
      "AI generated",
      "base"
    ]
  }
}

similarity_score = compare_semantic_similarity_v2(sample_run, sample_example)
print(f"Pontuação de similaridade semântica: {similarity_score}")